In [5]:
import os
import yaml
import pickle
import numpy as np
import pandas as pd
import networkx as nx
import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.utils import from_networkx, train_test_split_edges, negative_sampling
from torch_geometric.nn import SAGEConv

print("Imports OK")

# %% Charger configuration
with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

CACHE_PATH = config['graph']['cache_path']
PARTITION_PATH = "../results/communities/louvain_partition.pkl"
MODEL_OUTPUT = config['graph']['MODEL_PATH']

print("Configuration chargée")


Imports OK
Configuration chargée


In [6]:
# %% Chargement du graphe
with open(CACHE_PATH, 'rb') as f:
    G_nx = pickle.load(f)

print(f"Graphe chargé : {G_nx.number_of_nodes()} nœuds, {G_nx.number_of_edges()} arêtes")

# Charger Louvain
with open(PARTITION_PATH, "rb") as f:
    partition = pickle.load(f)

print(f"Communautés chargées : {len(set(partition.values()))}")


Graphe chargé : 10363 nœuds, 300869 arêtes
Communautés chargées : 45


In [7]:
import torch
from torch_geometric.data import Data
import networkx as nx

print("🔄 Conversion NetworkX → PyTorch Geometric (avec remapping)...\n")

# ===== ÉTAPE 1 : CRÉER UN MAPPING CONTINU =====
node_list = list(G_nx.nodes())
node_to_idx = {node: idx for idx, node in enumerate(node_list)}
print(f"✓ Mapping créé pour {len(node_list)} nœuds")
print(f"  Exemple: nœud {node_list[0]} → index 0")
print(f"  Exemple: nœud {node_list[-1]} → index {len(node_list)-1}")

# ===== ÉTAPE 2 : CONVERTIR LES ARÊTES =====
edges_remapped = [
    (node_to_idx[u], node_to_idx[v]) 
    for u, v in G_nx.edges()
]
edge_index = torch.tensor(edges_remapped, dtype=torch.long).t().contiguous()

print(f"\n✓ edge_index créé: {edge_index.shape}")
print(f"  Min index: {edge_index.min().item()}")
print(f"  Max index: {edge_index.max().item()}")
print(f"  Attendu max: {len(node_list) - 1}")

# Vérification critique
assert edge_index.max().item() < len(node_list), \
    f"❌ ERREUR: edge_index contient {edge_index.max().item()} mais seulement {len(node_list)} nœuds!"
assert edge_index.min().item() >= 0, \
    f"❌ ERREUR: edge_index contient des indices négatifs!"

# ===== ÉTAPE 3 : FEATURES (avec remapping) =====
print("\nCalcul des features...")
deg = dict(G_nx.degree())
pagerank = nx.pagerank(G_nx, max_iter=50)
clustering = nx.clustering(G_nx)

x = torch.tensor([
    [deg[node], pagerank[node], clustering[node]] 
    for node in node_list  # ← Utiliser node_list dans l'ordre
], dtype=torch.float)

print(f"✓ Features créées: {x.shape}")

# ===== ÉTAPE 4 : COMMUNAUTÉS (avec remapping) =====
community = torch.tensor(
    [partition.get(node, -1) for node in node_list],
    dtype=torch.long
)

missing = (community == -1).sum().item()
if missing > 0:
    print(f"⚠️ {missing} nœuds sans communauté (assignés à -1)")

# ===== ÉTAPE 5 : CRÉER L'OBJET DATA =====
data = Data(
    x=x,
    edge_index=edge_index,
    community=community,
    num_nodes=len(node_list)
)

print("\n" + "="*60)
print("📊 OBJET DATA CRÉÉ ET VÉRIFIÉ")
print("="*60)
print(f"  Nœuds: {data.num_nodes}")
print(f"  Arêtes: {data.edge_index.shape[1]}")
print(f"  Features: {data.x.shape}")
print(f"  edge_index range: [{edge_index.min().item()}, {edge_index.max().item()}]")
print(f"  Communautés: {community.max().item() + 1}")
print("="*60)

# ===== VÉRIFICATION FINALE =====
assert data.edge_index.max() < data.num_nodes, "❌ Indices hors limites!"
print("\n✅ Toutes les vérifications passées - Prêt pour le split!\n")

🔄 Conversion NetworkX → PyTorch Geometric (avec remapping)...

✓ Mapping créé pour 10363 nœuds
  Exemple: nœud 50562026 → index 0
  Exemple: nœud 2115828967 → index 10362

✓ edge_index créé: torch.Size([2, 300869])
  Min index: 0
  Max index: 10362
  Attendu max: 10362

Calcul des features...
✓ Features créées: torch.Size([10363, 3])

📊 OBJET DATA CRÉÉ ET VÉRIFIÉ
  Nœuds: 10363
  Arêtes: 300869
  Features: torch.Size([10363, 3])
  edge_index range: [0, 10362]
  Communautés: 45

✅ Toutes les vérifications passées - Prêt pour le split!



In [8]:
# %% Split edges (VERSION CORRIGÉE)
from torch_geometric.transforms import RandomLinkSplit

print(f"Avant split - edge_index: {data.edge_index.shape}")

transform = RandomLinkSplit(
    num_val=0.05,
    num_test=0.1,
    is_undirected=True,
    add_negative_train_samples=True,
    neg_sampling_ratio=1.0
)

train_data, val_data, test_data = transform(data)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_data = train_data.to(device)
val_data = val_data.to(device)
test_data = test_data.to(device)

print("\n✅ Train/val/test split OK")
print(f"  Train edges: {train_data.edge_index.size(1)}")
print(f"  Val edges: {val_data.edge_label_index.size(1)}")      # ← CORRIGÉ ICI
print(f"  Test edges: {test_data.edge_label_index.size(1)}")    # ← CORRIGÉ ICI
print(f"  Device: {device}")

Avant split - edge_index: torch.Size([2, 300869])

✅ Train/val/test split OK
  Train edges: 511480
  Val edges: 30086
  Test edges: 60172
  Device: cpu


In [9]:
# %% Model définition (VERSION AMÉLIORÉE)
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

class GraphSAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.5):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)
        self.dropout = dropout
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)  # ← Dropout ajouté
        x = self.conv2(x, edge_index)
        return x

class EdgePredictor(nn.Module):
    def __init__(self, embedding_dim):
        super().__init__()
        # Option 1 : Concaténation (votre version)
        self.lin = nn.Linear(embedding_dim * 2, 1)
    
    def forward(self, z, edge_index):
        src = z[edge_index[0]]
        dst = z[edge_index[1]]
        # Concaténation
        out = torch.cat([src, dst], dim=1)
        return torch.sigmoid(self.lin(out)).squeeze()  # ← squeeze() ajouté

# OU version alternative avec produit scalaire (plus simple et souvent meilleure)
class EdgePredictorDot(nn.Module):
    """Prédiction par produit scalaire (recommandé)"""
    def __init__(self):
        super().__init__()
    
    def forward(self, z, edge_index):
        src = z[edge_index[0]]
        dst = z[edge_index[1]]
        # Produit scalaire
        return torch.sigmoid((src * dst).sum(dim=-1))

# Initialisation
embedding_dim = 64
gnn = GraphSAGE(
    in_channels=train_data.x.size(1),  # 3 features
    hidden_channels=128,
    out_channels=embedding_dim,
    dropout=0.5
).to(device)

edge_predictor = EdgePredictor(embedding_dim).to(device)
# OU
# edge_predictor = EdgePredictorDot().to(device)  # Version plus simple

print("✅ Modèles créés")
print(f"  GNN params: {sum(p.numel() for p in gnn.parameters()):,}")
print(f"  Edge predictor params: {sum(p.numel() for p in edge_predictor.parameters()):,}")

✅ Modèles créés
  GNN params: 17,344
  Edge predictor params: 129


In [10]:
# %% Init model
encoder = GraphSAGE(
    in_channels=train_data.x.size(1),  # ← Utiliser train_data, pas data
    hidden_channels=128,
    out_channels=64,
    dropout=0.5
).to(device)

predictor = EdgePredictor(64).to(device)  # ← Adapter à out_channels

optimizer = torch.optim.Adam(
    list(encoder.parameters()) + list(predictor.parameters()),
    lr=0.01,
    weight_decay=5e-4  # ← Régularisation
)

criterion = nn.BCELoss()  # Loss pour classification binaire

print("✅ Modèles initialisés")
print(f"  Encoder: {sum(p.numel() for p in encoder.parameters()):,} params")
print(f"  Predictor: {sum(p.numel() for p in predictor.parameters()):,} params")

✅ Modèles initialisés
  Encoder: 17,344 params
  Predictor: 129 params


In [11]:
print("📊 Analyse de la distribution des degrés...\n")

# Calculer les degrés
node_degrees = dict(G_nx.degree())
degrees_array = np.array([node_degrees[node] for node in node_list])

# Calculer percentiles
degree_p50 = np.percentile(degrees_array, 50)
degree_p90 = np.percentile(degrees_array, 90)
degree_p95 = np.percentile(degrees_array, 95)

print(f"Distribution des degrés:")
print(f"   Médiane (p50): {degree_p50:.0f}")
print(f"   p90: {degree_p90:.0f}")
print(f"   p95 (seuil hubs): {degree_p95:.0f}")
print(f"   Max: {degrees_array.max():.0f}")

# Identifier les nœuds par catégorie
low_degree_mask = degrees_array <= degree_p50
mid_degree_mask = (degrees_array > degree_p50) & (degrees_array <= degree_p90)
high_degree_mask = (degrees_array > degree_p90) & (degrees_array <= degree_p95)
extreme_hub_mask = degrees_array > degree_p95

low_nodes = np.where(low_degree_mask)[0]
mid_nodes = np.where(mid_degree_mask)[0]
high_nodes = np.where(high_degree_mask)[0]
extreme_hubs = np.where(extreme_hub_mask)[0]

print(f"\nCatégorisation:")
print(f"   Low (≤p50): {len(low_nodes)} nœuds")
print(f"   Mid (p50-p90): {len(mid_nodes)} nœuds")
print(f"   High (p90-p95): {len(high_nodes)} nœuds")
print(f"   Extreme (>p95): {len(extreme_hubs)} nœuds")
print(f"\n⚠️ Les extreme hubs seront EXCLUS du negative sampling")


📊 Analyse de la distribution des degrés...

Distribution des degrés:
   Médiane (p50): 11
   p90: 278
   p95 (seuil hubs): 312
   Max: 568

Catégorisation:
   Low (≤p50): 5263 nœuds
   Mid (p50-p90): 4277 nœuds
   High (p90-p95): 314 nœuds
   Extreme (>p95): 509 nœuds

⚠️ Les extreme hubs seront EXCLUS du negative sampling


In [12]:
def train_with_stratified_negatives():
    """Training avec negative sampling stratifié pour éviter biais hub"""
    encoder.train()
    predictor.train()
    optimizer.zero_grad()
    
    # Encoder le graphe
    z = encoder(train_data.x, train_data.edge_index)
    
    # ===== NEGATIVE SAMPLING STRATIFIÉ =====
    num_pos = train_data.edge_label_index[:, train_data.edge_label == 1].size(1)
    
    # Ratio: 50% low, 30% mid, 20% high (pas d'extreme hubs)
    num_low = int(num_pos * 0.5)
    num_mid = int(num_pos * 0.3)
    num_high = int(num_pos * 0.2)
    
    # Sources aléatoires
    src_low = torch.randint(0, train_data.num_nodes, (num_low,), device=device)
    src_mid = torch.randint(0, train_data.num_nodes, (num_mid,), device=device)
    src_high = torch.randint(0, train_data.num_nodes, (num_high,), device=device)
    
    # Targets stratifiés (SANS extreme hubs)
    dst_low = torch.from_numpy(np.random.choice(low_nodes, num_low, replace=True)).to(device)
    dst_mid = torch.from_numpy(np.random.choice(mid_nodes, num_mid, replace=True)).to(device)
    dst_high = torch.from_numpy(np.random.choice(high_nodes, num_high, replace=True)).to(device)
    
    # Créer edge_index négatif
    neg_src = torch.cat([src_low, src_mid, src_high])
    neg_dst = torch.cat([dst_low, dst_mid, dst_high])
    neg_edge_index = torch.stack([neg_src, neg_dst])
    
    # ===== PRÉDICTIONS =====
    pos_edge_index = train_data.edge_label_index[:, train_data.edge_label == 1]
    pos_pred = predictor(z, pos_edge_index)
    neg_pred = predictor(z, neg_edge_index)
    
    # ===== LOSS AVEC PONDÉRATION =====
    # Pénaliser plus les erreurs sur low-degree nodes
    pos_target_degrees = degrees_array[pos_edge_index[1].cpu().numpy()]
    pos_weights = torch.from_numpy(1.0 / np.log(pos_target_degrees + 2)).float().to(device)
    pos_weights = pos_weights / pos_weights.sum() * len(pos_weights)
    
    pos_loss = F.binary_cross_entropy(pos_pred, torch.ones_like(pos_pred), weight=pos_weights)
    neg_loss = F.binary_cross_entropy(neg_pred, torch.zeros_like(neg_pred))
    
    loss = pos_loss + neg_loss
    
    loss.backward()
    
    # Gradient clipping
    torch.nn.utils.clip_grad_norm_(encoder.parameters(), max_norm=1.0)
    torch.nn.utils.clip_grad_norm_(predictor.parameters(), max_norm=1.0)
    
    optimizer.step()
    
    return loss.item(), pos_pred.mean().item(), neg_pred.mean().item()

print("✅ Fonction d'entraînement créée (avec stratification anti-hub)")



✅ Fonction d'entraînement créée (avec stratification anti-hub)


In [13]:
from sklearn.metrics import roc_auc_score

@torch.no_grad()
def test_with_degree_analysis(data_split):
    """Test avec analyse par catégorie de degré"""
    encoder.eval()
    predictor.eval()
    
    z = encoder(data_split.x, train_data.edge_index)
    pred = predictor(z, data_split.edge_label_index)
    
    pred_cpu = pred.cpu().numpy()
    label_cpu = data_split.edge_label.cpu().numpy()
    
    # Métriques globales
    auc = roc_auc_score(label_cpu, pred_cpu)
    pred_binary = (pred > 0.5).cpu().numpy()
    acc = (pred_binary == label_cpu).mean()
    
    # Analyse par degré
    target_degrees = degrees_array[data_split.edge_label_index[1].cpu().numpy()]
    
    low_mask = target_degrees <= degree_p50
    high_mask = target_degrees > degree_p90
    
    auc_low = roc_auc_score(label_cpu[low_mask], pred_cpu[low_mask]) if low_mask.sum() > 10 else None
    auc_high = roc_auc_score(label_cpu[high_mask], pred_cpu[high_mask]) if high_mask.sum() > 10 else None
    
    return auc, acc, auc_low, auc_high

print("✅ Fonction de test créée (avec analyse par degré)")


✅ Fonction de test créée (avec analyse par degré)


In [14]:
print("🚀 Début de l'entraînement (anti-hub bias)...\n")

best_val_auc = 0
patience = 25
patience_counter = 0
history = {
    'train_loss': [], 
    'val_auc': [], 
    'val_auc_low': [], 
    'val_auc_high': []
}

for epoch in range(1, 201):
    loss, pos_mean, neg_mean = train_with_stratified_negatives()
    history['train_loss'].append(loss)
    
    if epoch % 5 == 0:
        val_auc, val_acc, val_auc_low, val_auc_high = test_with_degree_analysis(val_data)
        
        history['val_auc'].append(val_auc)
        history['val_auc_low'].append(val_auc_low if val_auc_low else 0)
        history['val_auc_high'].append(val_auc_high if val_auc_high else 0)
        
        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Val AUC: {val_auc:.4f} | "
              f"Pos: {pos_mean:.3f} | Neg: {neg_mean:.3f}")
        
        if val_auc_low and val_auc_high:
            print(f"           | AUC Low-deg: {val_auc_low:.4f} | AUC High-deg: {val_auc_high:.4f}")
        
        # Early stopping
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            patience_counter = 0
            
            torch.save({
                'encoder': encoder.state_dict(),
                'predictor': predictor.state_dict(),
                'val_auc': val_auc,
                'epoch': epoch
            }, 'best_model.pt')
            print(f"           | ✅ Meilleur modèle sauvegardé")
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            print(f"\n⚠️ Early stopping à l'epoch {epoch}")
            break

print("\n✅ Entraînement terminé!")



🚀 Début de l'entraînement (anti-hub bias)...

Epoch 005 | Loss: 75.5039 | Val AUC: 0.2366 | Pos: 0.000 | Neg: 0.001
           | AUC Low-deg: 0.6727 | AUC High-deg: 0.0748
           | ✅ Meilleur modèle sauvegardé
Epoch 010 | Loss: 29.2397 | Val AUC: 0.8332 | Pos: 0.856 | Neg: 0.610
           | AUC Low-deg: 0.6811 | AUC High-deg: 0.3993
           | ✅ Meilleur modèle sauvegardé
Epoch 015 | Loss: 29.4014 | Val AUC: 0.8381 | Pos: 0.938 | Neg: 0.510
           | AUC Low-deg: 0.6818 | AUC High-deg: 0.5067
           | ✅ Meilleur modèle sauvegardé
Epoch 020 | Loss: 27.1053 | Val AUC: 0.8724 | Pos: 0.965 | Neg: 0.684
           | AUC Low-deg: 0.6756 | AUC High-deg: 0.5000
           | ✅ Meilleur modèle sauvegardé
Epoch 025 | Loss: 13.2318 | Val AUC: 0.8231 | Pos: 0.221 | Neg: 0.337
           | AUC Low-deg: 0.4189 | AUC High-deg: 0.9737
Epoch 030 | Loss: 10.5426 | Val AUC: 0.2755 | Pos: 0.409 | Neg: 0.268
           | AUC Low-deg: 0.4965 | AUC High-deg: 0.9630
Epoch 035 | Loss: 13.0520 | Va

In [15]:
print("📊 Évaluation sur le set de test...\n")

# Charger meilleur modèle
checkpoint = torch.load('best_model.pt')
encoder.load_state_dict(checkpoint['encoder'])
predictor.load_state_dict(checkpoint['predictor'])

# Test final
test_auc, test_acc, test_auc_low, test_auc_high = test_with_degree_analysis(test_data)

print(f"{'='*60}")
print(f"RÉSULTATS FINAUX")
print(f"{'='*60}")
print(f"  Best Val AUC: {best_val_auc:.4f}")
print(f"  Test AUC (global): {test_auc:.4f}")
print(f"  Test Acc: {test_acc:.4f}")
if test_auc_low:
    print(f"  Test AUC (low-degree): {test_auc_low:.4f}")
if test_auc_high:
    print(f"  Test AUC (high-degree): {test_auc_high:.4f}")
print(f"{'='*60}")


📊 Évaluation sur le set de test...

RÉSULTATS FINAUX
  Best Val AUC: 0.9143
  Test AUC (global): 0.9123
  Test Acc: 0.8498
  Test AUC (low-degree): 0.4903
  Test AUC (high-degree): 0.9686


In [16]:
print("\n🔬 Diagnostic des embeddings...\n")

encoder.eval()
with torch.no_grad():
    final_embeddings = encoder(data.x.to(device), data.edge_index.to(device))

# Analyser similarité des hubs
hub_indices = extreme_hubs[:min(10, len(extreme_hubs))]
hub_embeds = final_embeddings[hub_indices].cpu().numpy()

from sklearn.metrics.pairwise import cosine_similarity
hub_sim = cosine_similarity(hub_embeds)
np.fill_diagonal(hub_sim, 0)
avg_hub_sim = hub_sim.mean()

print(f"Similarité moyenne entre hubs: {avg_hub_sim:.4f}")
if avg_hub_sim > 0.9:
    print(f"⚠️ Embeddings des hubs trop similaires (risque de biais)")
    print(f"   → Considérer : plus de dropout, plus d'epochs, ou moins de hidden_channels")
else:
    print(f"✅ Embeddings des hubs bien différenciés")

# Analyser tous les embeddings
all_embeds = final_embeddings.cpu().numpy()
all_sim = cosine_similarity(all_embeds[:100])  # Échantillon
np.fill_diagonal(all_sim, 0)
avg_all_sim = all_sim.mean()

print(f"\nSimilarité moyenne (échantillon global): {avg_all_sim:.4f}")
print(f"Écart-type des embeddings: {all_embeds.std():.4f}")




🔬 Diagnostic des embeddings...

Similarité moyenne entre hubs: 0.6638
✅ Embeddings des hubs bien différenciés

Similarité moyenne (échantillon global): 0.5727
Écart-type des embeddings: 1.6200


In [17]:
print("\n💾 Sauvegarde COMPLÈTE du modèle...\n")

torch.save({
    'model_state': {
        'encoder': encoder.state_dict(),
        'predictor': predictor.state_dict()
    },
    'embeddings': final_embeddings.cpu(),
    'node_to_idx': node_to_idx,
    'metrics': {
        'val_auc': best_val_auc,
        'test_auc': test_auc,
        'test_acc': test_acc,
        'test_auc_low': test_auc_low if test_auc_low else 0,
        'test_auc_high': test_auc_high if test_auc_high else 0
    },
    'hyperparameters': {
        'in_channels': train_data.x.size(1),
        'hidden_channels': 128,
        'embedding_dim': 64,
        'dropout': 0.5
    },
    'degree_stats': {
        'p50': float(degree_p50),
        'p90': float(degree_p90),
        'p95': float(degree_p95),
        'max': float(degrees_array.max())
    },
    'metadata': {
        'num_nodes': len(node_to_idx),
        'num_edges': data.edge_index.shape[1],
        'training_date': str(pd.Timestamp.now()),
        'device': str(device),
        'hub_embedding_similarity': float(avg_hub_sim),
        'global_embedding_similarity': float(avg_all_sim),
        'training_method': 'stratified_negative_sampling_v2'
    }
}, MODEL_OUTPUT)

print(f"✅ Modèle sauvegardé: {MODEL_OUTPUT}")



💾 Sauvegarde COMPLÈTE du modèle...

✅ Modèle sauvegardé: ../results/link_prediction_model.pt


In [18]:
print("\n🧪 Vérification du fichier sauvegardé...\n")

test_checkpoint = torch.load(MODEL_OUTPUT, map_location='cpu', weights_only=False)

print(f"✅ Fichier valide:")
print(f"   Embeddings: {test_checkpoint['embeddings'].shape}")
print(f"   Nœuds: {len(test_checkpoint['node_to_idx'])}")
print(f"   Test AUC: {test_checkpoint['metrics']['test_auc']:.4f}")
print(f"   Hub similarity: {test_checkpoint['metadata']['hub_embedding_similarity']:.4f}")
print(f"   Training method: {test_checkpoint['metadata']['training_method']}")



🧪 Vérification du fichier sauvegardé...

✅ Fichier valide:
   Embeddings: torch.Size([10363, 64])
   Nœuds: 10363
   Test AUC: 0.9123
   Hub similarity: 0.6638
   Training method: stratified_negative_sampling_v2


In [19]:
print("\n🧪 Test de prédictions sur quelques exemples...\n")

# Prendre un chercheur aléatoire
test_author_idx = np.random.choice(len(node_list))
test_author_id = node_list[test_author_idx]

print(f"Chercheur test: {test_author_id} (degree: {node_degrees[test_author_id]})")

# Prendre 10 candidats aléatoires
candidate_indices = np.random.choice(len(node_list), 10, replace=False)

# Prédire
encoder.eval()
with torch.no_grad():
    z = encoder(data.x.to(device), data.edge_index.to(device))
    
    edge_index = torch.tensor([
        [test_author_idx] * 10,
        candidate_indices.tolist()
    ], dtype=torch.long, device=device)
    
    scores = predictor(z, edge_index).cpu().numpy()

print(f"\nTop 5 scores:")
top_5_idx = np.argsort(scores)[-5:][::-1]
for rank, idx in enumerate(top_5_idx, 1):
    cand_id = node_list[candidate_indices[idx]]
    cand_deg = node_degrees[cand_id]
    print(f"  {rank}. Candidat {cand_id} (degree: {cand_deg}) → score: {scores[idx]:.4f}")

print("\n✅ Test de prédiction OK")




🧪 Test de prédictions sur quelques exemples...

Chercheur test: 2064770039 (degree: 35)

Top 5 scores:
  1. Candidat 48650879 (degree: 128) → score: 0.4601
  2. Candidat 2046135 (degree: 9) → score: 0.3702
  3. Candidat 1700880 (degree: 9) → score: 0.2607
  4. Candidat 39393520 (degree: 12) → score: 0.2534
  5. Candidat 2302053110 (degree: 7) → score: 0.1980

✅ Test de prédiction OK


In [20]:
print("\n📋 Création d'exemples pour tests API...\n")

# Top chercheurs par degré
top_by_degree = sorted(node_degrees.items(), key=lambda x: x[1], reverse=True)[:20]

# Échantillon aléatoire
random_sample = np.random.choice(node_list, min(10, len(node_list)), replace=False)

example_nodes = {
    'top_connected': [str(node) for node, deg in top_by_degree[:10]],
    'random_sample': [str(node) for node in random_sample],
    'low_degree': [str(node) for node in node_list if node_degrees[node] < degree_p50][:10],
    'mid_degree': [str(node) for node in node_list if degree_p50 < node_degrees[node] < degree_p90][:10]
}

examples_path = "../results/example_researchers.json"
import json
with open(examples_path, 'w') as f:
    json.dump(example_nodes, indent=2, fp=f)

print(f"✅ Exemples sauvegardés: {examples_path}")
print(f"\nTop 5 chercheurs pour tester:")
for i, (node, deg) in enumerate(top_by_degree[:5], 1):
    print(f"   {i}. {node} ({deg} collaborations)")

print(f"\n{'='*60}")
print(f"✅ PRÊT POUR L'API!")
print(f"{'='*60}")
print(f"\nFichiers créés:")
print(f"   {MODEL_OUTPUT}")
print(f"   {PARTITION_PATH}")
print(f"   {examples_path}")
print(f"{'='*60}")


📋 Création d'exemples pour tests API...

✅ Exemples sauvegardés: ../results/example_researchers.json

Top 5 chercheurs pour tester:
   1. 46617804 (568 collaborations)
   2. 39589154 (535 collaborations)
   3. 1727524 (531 collaborations)
   4. 2010057 (528 collaborations)
   5. 2322150 (525 collaborations)

✅ PRÊT POUR L'API!

Fichiers créés:
   ../results/link_prediction_model.pt
   ../results/communities/louvain_partition.pkl
   ../results/example_researchers.json
